```{=latex}
\usepackage{hyperref}
\usepackage{graphicx}
\usepackage{listings}
\usepackage{textcomp}
\usepackage{fancyvrb}

\newcommand{\passthrough}[1]{\lstset{mathescape=false}#1\lstset{mathescape=true}}
\newcommand{\tightlist}{}
```

```{=latex}
\title{Best Practices for CI in Python}
\author{Moshe Zadka -- https://cobordism.com}
\date{}

\begin{document}
\begin{titlepage}
\maketitle
\end{titlepage}

\frame{\titlepage}
```

```{=latex}
\begin{frame}
\frametitle{Acknowledgement of Country}

Belmont (in San Francisco Bay Area Peninsula)

Ancestral homeland of the Ramaytush Ohlone people

\end{frame}
```

## What is CI?

* Local build (lint, test, package)
* Server build: nightly, main, patch, exotica
* Focus of talk: Patch
* Runners
* Logs


## What is Good CI?

* Accuracy
* Actionability
* Promptness
* Cost


## Improving accuracy

* Containers
* Pinning versions
* Test quality

## Improving actionability

* Verbose tests
* Verbose failures
* Environmental details

## Improving promptness

* Cache
* Pool
* Fail fast
* Parallelize

## Improving cost

* Kill useless runs
* Stop runs early
* Optimize tests

## Summary

* Decide trade-offs
* Measure
* Improve
* Repeat

```{=latex}
\end{document}
```